In [1]:
import pandas as pd
import re
from pykakasi import kakasi
import jaconv

In [2]:
df = pd.read_csv('japanese_lemmas.csv')

In [3]:
df

,rank,frequency,lemma
0,1,41309.50,の
1,2,23509.54,に
2,3,22216.80,は
3,4,20431.93,て
4,5,20326.59,を
...,...,...,...
14995,14996,2.24,夕べ
14996,14997,2.24,売場
14997,14998,2.24,たたき台
14998,14999,2.24,かしこ


In [4]:
jlpt = pd.read_csv('JLPTComplete.csv')

In [5]:
jlpt

,id,kanji,kana,hiragana,level,haskanji
0,0,会う,あう,あう,n5,True
1,1,青,あお,あお,n5,True
2,2,青い,あおい,あおい,n5,True
3,3,赤,あか,あか,n5,True
4,4,赤い,あかい,あかい,n5,True
...,...,...,...,...,...,...
7029,7029,和風,わふう,わふう,n1,True
7030,7030,和文,わぶん,わぶん,n1,True
7031,7031,藁,わら,わら,n1,True
7032,7032,割り込む,わりこむ,わりこむ,n1,True


In [ ]:
#Remove symbols
#Take out one one character words that are only Katakana or only

In [6]:
def contains_kana_or_kanji(text):
    # Regular expression to match kana and kanji characters
    pattern = re.compile(r'[ぁ-んァ-ン一-龯]')
    return bool(pattern.search(text))

In [7]:
kana_kanji = df[df['lemma'].apply(contains_kana_or_kanji)]

In [8]:
kana_kanji

,rank,frequency,lemma
0,1,41309.50,の
1,2,23509.54,に
2,3,22216.80,は
3,4,20431.93,て
4,5,20326.59,を
...,...,...,...
14995,14996,2.24,夕べ
14996,14997,2.24,売場
14997,14998,2.24,たたき台
14998,14999,2.24,かしこ


In [9]:
def is_single_kanji(character):
    # Check if the character has only one character
    if len(character) != 1:
        return True
    else:
        # Check if the character is a kanji
        return '\u4e00' <= character <= '\u9faf'

In [10]:
kana_kanji = kana_kanji[kana_kanji['lemma'].apply(is_single_kanji)]

In [11]:
kana_kanji = kana_kanji.reset_index()
del kana_kanji['index']

In [12]:
kakasi = kakasi()
kakasi.setMode('J','H')

C:\Users\rgova\AppData\Local\Temp\ipykernel_10776\681701937.py:2: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi.setMode('J','H')


In [13]:
conv = kakasi.getConverter()

C:\Users\rgova\AppData\Local\Temp\ipykernel_10776\2787279695.py:1: DeprecationWarning: Call to deprecated method getConverter. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  conv = kakasi.getConverter()


In [14]:
def convert_kanji_to_hiragana(text):
    has_kanji = False
    converted_text = ""
    
    for char in text:
        # Check if the character is a kanji
        if '\u4e00' <= char <= '\u9faf':
            has_kanji = True
            # Convert kanji to hiragana
            converted_text = conv.do(text)
            break
    
    # If no kanji found, return the original text, else return the converted text
    return converted_text if has_kanji else text

In [15]:
kana_kanji['kana'] = kana_kanji['lemma'].apply(convert_kanji_to_hiragana)

C:\Users\rgova\AppData\Local\Temp\ipykernel_10776\4094959480.py:10: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  converted_text = conv.do(text)


In [24]:
kana_kanji

,rank,frequency,kanji,kana,hiragana,haskanji
0,9,16841.17,する,する,する,False
1,11,9604.49,ます,ます,ます,False
2,13,8189.00,ない,ない,ない,False
3,14,8140.22,いる,いる,いる,False
4,16,6766.19,ある,ある,ある,False
...,...,...,...,...,...,...
14681,14996,2.24,夕べ,ゆうべ,ゆうべ,True
14682,14997,2.24,売場,うりば,うりば,True
14683,14998,2.24,たたき台,たたきだい,たたきだい,True
14684,14999,2.24,かしこ,かしこ,かしこ,False


In [25]:
jlpt

,id,kanji,kana,hiragana,level,haskanji
0,0,会う,あう,あう,n5,True
1,1,青,あお,あお,n5,True
2,2,青い,あおい,あおい,n5,True
3,3,赤,あか,あか,n5,True
4,4,赤い,あかい,あかい,n5,True
...,...,...,...,...,...,...
7029,7029,和風,わふう,わふう,n1,True
7030,7030,和文,わぶん,わぶん,n1,True
7031,7031,藁,わら,わら,n1,True
7032,7032,割り込む,わりこむ,わりこむ,n1,True


In [18]:
def convert_katakana_to_hiragana(text):
    return jaconv.kata2hira(text)

In [19]:
kana_kanji['hirakana'] = kana_kanji['kana'].apply(convert_katakana_to_hiragana)

In [20]:
def contains_kanji(word):
    kanji_pattern = re.compile(r'[\u4e00-\u9faf]')
    return bool(kanji_pattern.search(word))

In [21]:
kana_kanji['haskanji'] = kana_kanji['lemma'].apply(contains_kanji)

In [22]:
column_mapping = {'lemma':'kanji',
                 'hirakana':'hiragana'}

In [23]:
kana_kanji = kana_kanji.rename(columns=column_mapping)

In [ ]:
word_columns = ['kanji', 'kana', 'hiragana', 'haskanji']
tag_columns = ['wordsetid','wordid']

all_words_df = pd.DataFrame(columns=word_columns)
tag_df = pd.Dataframe(columns=tag_columns)

In [ ]:
for idx, r in kana_kanji.iterrows():
    #TODO loop through both dataframes, add any new words (based on kanji) to new dataframe
    #For every word need to add a tag in the tag dataframe
    if kana_kanji['kanji'].iloc[idx] is not in all_words_df['kanji'].values:

In [26]:
jlpt.to_csv('jlpt_words.csv')

In [27]:
kana_kanji.to_csv('freq_words.csv')

In [53]:
convert_katakana_to_hiragana(kana_kanji['lemma'].iloc[29])

'思う'